In [1]:
from __future__ import division
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pyspan.config import *
from pyspan.plurals.analysis import *
assert mturk

/Users/sabinasloman/Box/LoP/pyspan/pyspan/config.py:19: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/Users/sabinasloman/.pyenv/versions/2.7.17/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/sabinasloman/.pyenv/versions/2.7.17/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/sabinasloman/.pyenv/versions/2.7.17/envs/lop_env/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/sabinasloman/.pyenv/versions/2.7.17/envs/lop_env/lib/python2.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  

## Logistic regression

Selection of plural ~ Condition + Dummy indicating whether or not this was the first survey the participant took (including participant-level effects)

We will commit to throwing out all data from the construal level survey by participants who did not take the survey first if the order dummy has a non-zero coefficient.

In [2]:
cdummied, Y = dummy(cl, sets = np.stack((words["large"].values,
                                         words["small"].values)),
                    classes = [ "ABSTRACT", "CONCRETE" ])
X, Y = df_to_matrix(cdummied, Y, 
                    columns = { 0: "condition", 
                                1: "order" })

In [3]:
logit = SparseLR(Y, X); print logit.coef[:2]; logit.auc

/Users/sabinasloman/.pyenv/versions/2.7.17/envs/lop_env/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2052: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


[0.55567561 0.        ]


0.6123788019308914

## t-tests

We predict that participants in the ABSTRACT condition are more likely to select the pluralized form of the item than participants in the CONCRETE condition.

In [4]:
csummary = cl[["Condition"]]
dat = cl[ixs].values
props = np.apply_along_axis(get_prop, 1, dat, 
                            words["large"], 
                            words["small"])
csummary["ppl"] = props
assert csummary.values.shape == (len(cl), 2)

/Users/sabinasloman/.pyenv/versions/2.7.17/envs/lop_env/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
a = csummary.loc[csummary["Condition"] == "ABSTRACT"]["ppl"].values
b = csummary.loc[csummary["Condition"] == "CONCRETE"]["ppl"].values
stats.ttest_ind(a, b)

Ttest_indResult(statistic=5.854710200421765, pvalue=1.3444795767615869e-08)